## *Movie Reviews Part 2*
***

In [165]:
#system specific
import os
import sys
assert sys.version_info >= (3,5)

#import the libraries
import pandas as pd
import numpy as np
import random
#visualizations 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#consistent size plots
from pylab import rcParams
rcParams['figure.figsize'] = (12,5)
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.labelsize'] = 12

#handle unwanted warnings
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)


In [166]:
#Load the data
master_data = pd.read_csv('master.csv',delimiter=',',engine='python')

In [167]:
master_data.head()

,MovieID,Title,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Count,Genre_1,Genre_2
0,1,Toy Story (1995),1,5,978824268,F,1,10,48067,1,Animation,Children's
1,48,Pocahontas (1995),1,5,978824351,F,1,10,48067,1,Animation,Children's
2,150,Apollo 13 (1995),1,5,978301777,F,1,10,48067,1,Drama,Drama
3,260,Star Wars: Episode IV - A New Hope (1977),1,4,978300760,F,1,10,48067,1,Action,Adventure
4,527,Schindler's List (1993),1,5,978824195,F,1,10,48067,1,Drama,War


In [168]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 12 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   MovieID     1000209 non-null  int64 
 1   Title       1000209 non-null  object
 2   UserID      1000209 non-null  int64 
 3   Rating      1000209 non-null  int64 
 4   Timestamp   1000209 non-null  int64 
 5   Gender      1000209 non-null  object
 6   Age         1000209 non-null  int64 
 7   Occupation  1000209 non-null  int64 
 8   Zip-code    1000209 non-null  object
 9   Count       1000209 non-null  int64 
 10  Genre_1     1000209 non-null  object
 11  Genre_2     1000209 non-null  object
dtypes: int64(7), object(5)
memory usage: 91.6+ MB


For further analysis of the data, the unique identifier MovieID will be used. The data type of this feature is integer. 

In [169]:
movies = master_data.copy()

In [170]:
#drop the title column
movies.drop('Title',axis=1,inplace=True)

In [171]:
#create a pivot table with all the user reviews
num_reviews = pd.pivot_table(master_data,values='Count',index='MovieID',aggfunc='sum')

In [172]:
max(num_reviews['Count'])

3428

In [173]:
#this is the American Beauty as seen in the previous notebook .. 
num_reviews[num_reviews['Count']==3428]

,Count
MovieID,
2858,3428


In [174]:
num_reviews.describe()

,Count
count,3706.000000
mean,269.889099
std,384.047838
min,1.000000
25%,33.000000
50%,123.500000
75%,350.000000
max,3428.000000


In [175]:
#create a dataframe where the number of reviews is less than the median number of reviews
less_reviewed = num_reviews[num_reviews['Count']< num_reviews['Count'].median()]

In [176]:
less_reviewed.head()

,Count
MovieID,
8,68
9,102
13,99
26,100
27,61


In [177]:
#the movie id's are in the index
less_reviewed_id = less_reviewed.index

In [178]:
#view a few of the MovieID's
less_reviewed_id[:8]

Int64Index([8, 9, 13, 26, 27, 30, 33, 35], dtype='int64', name='MovieID')

In [179]:
#drop all the rows where the MovieID is in less_reviewed_id
movies = movies[~movies['MovieID'].isin(less_reviewed_id)]

In [180]:
#check the new number of observations left
len(movies)

923889

In [181]:
#number of unique movies now in the dataset
len(movies['MovieID'].unique())

1853

Instead of over 3000 movies, we are now left with 1853 movies. All these movies have atleast the median value of the number of reviews. The assessment would be less biased now. 

### *Encode the Genres and Gender*

In [182]:
#label encode the Genres 
from sklearn.preprocessing import LabelEncoder

In [183]:
encoder =  LabelEncoder()
movies['Genre_enc_1'] = encoder.fit_transform(movies['Genre_1'])
movies['Genre_enc_2'] = encoder.fit_transform(movies['Genre_2'])

In [184]:
movies.head()

,MovieID,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Count,Genre_1,Genre_2,Genre_enc_1,Genre_enc_2
0,1,1,5,978824268,F,1,10,48067,1,Animation,Children's,2,3
1,48,1,5,978824351,F,1,10,48067,1,Animation,Children's,2,3
2,150,1,5,978301777,F,1,10,48067,1,Drama,Drama,7,7
3,260,1,4,978300760,F,1,10,48067,1,Action,Adventure,0,1
4,527,1,5,978824195,F,1,10,48067,1,Drama,War,7,16


In [185]:
#drop the redundant Genre_x columns
movies.drop(['Genre_1','Genre_2'],axis=1,inplace=True)

In [186]:
movies.columns

Index(['MovieID', 'UserID', 'Rating', 'Timestamp', 'Gender', 'Age',
       'Occupation', 'Zip-code', 'Count', 'Genre_enc_1', 'Genre_enc_2'],
      dtype='object')

In [187]:
#encode the Gender
movies['Gender'].unique()

array(['F', 'M'], dtype=object)

In [189]:
movies['Gender_enc'] = pd.get_dummies(movies['Gender'],drop_first=True)

In [190]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Count,Genre_enc_1,Genre_enc_2,Gender_enc
0,1,1,5,978824268,F,1,10,48067,1,2,3,0
1,48,1,5,978824351,F,1,10,48067,1,2,3,0


In [191]:
#drop the redundant Gender columns
movies.drop('Gender',axis=1,inplace=True)

In [192]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Zip-code,Count,Genre_enc_1,Genre_enc_2,Gender_enc
0,1,1,5,978824268,1,10,48067,1,2,3,0
1,48,1,5,978824351,1,10,48067,1,2,3,0


## *Feature Engineering*
### *Zipcode*

In [193]:
#replace zip code with only the first 3 digits
movies['Zip-code'].astype(str)

0          48067
1          48067
2          48067
3          48067
4          48067
           ...  
1000204    92843
1000205    92843
1000206    92843
1000207    92843
1000208    92843
Name: Zip-code, Length: 923889, dtype: object

In [194]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923889 entries, 0 to 1000208
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   MovieID      923889 non-null  int64 
 1   UserID       923889 non-null  int64 
 2   Rating       923889 non-null  int64 
 3   Timestamp    923889 non-null  int64 
 4   Age          923889 non-null  int64 
 5   Occupation   923889 non-null  int64 
 6   Zip-code     923889 non-null  object
 7   Count        923889 non-null  int64 
 8   Genre_enc_1  923889 non-null  int64 
 9   Genre_enc_2  923889 non-null  int64 
 10  Gender_enc   923889 non-null  uint8 
dtypes: int64(9), object(1), uint8(1)
memory usage: 78.4+ MB


In [195]:
#retain only the first 3 letters of the zip code to determine the area 
movies['Zip-code_area'] =  movies['Zip-code'].str[:3]

In [196]:
movies.head()

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Zip-code,Count,Genre_enc_1,Genre_enc_2,Gender_enc,Zip-code_area
0,1,1,5,978824268,1,10,48067,1,2,3,0,480
1,48,1,5,978824351,1,10,48067,1,2,3,0,480
2,150,1,5,978301777,1,10,48067,1,7,7,0,480
3,260,1,4,978300760,1,10,48067,1,0,1,0,480
4,527,1,5,978824195,1,10,48067,1,7,16,0,480


In [197]:
#drop the zip code
movies.drop('Zip-code',axis=1,inplace=True)

In [198]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,Zip-code_area
0,1,1,5,978824268,1,10,1,2,3,0,480
1,48,1,5,978824351,1,10,1,2,3,0,480


In [199]:
len(movies['Zip-code_area'].unique())

678

In [200]:
movies['Zip-code_area'].value_counts().sort_values(ascending=False)[:10]

554    23718
100    21477
551    19978
941    19532
021    19252
606    18457
981    16617
481    13725
921    13487
900    12445
Name: Zip-code_area, dtype: int64

The area code represented by the zip code are now in str format. Instead of label encoding these and created 678 different integer label, another option could be to replace with the sum of the digits. This way the scale of the feature will also be within acceptable limits.

In [201]:
def sum_digits(num):
    '''This function returns the sum of digits'''
    digit_sum = 0
    for digit in str(num):
        digit_sum += int(digit)
    return digit_sum

In [202]:
#feature with sum of digits of the are zip code
movies['zipcode_sum'] = movies['Zip-code_area'].apply(sum_digits)

In [203]:
movies.head(3)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,Zip-code_area,zipcode_sum
0,1,1,5,978824268,1,10,1,2,3,0,480,12
1,48,1,5,978824351,1,10,1,2,3,0,480,12
2,150,1,5,978301777,1,10,1,7,7,0,480,12


In [204]:
#drop the earlier zip code
movies.drop('Zip-code_area',axis=1,inplace=True)

In [205]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,zipcode_sum
0,1,1,5,978824268,1,10,1,2,3,0,12
1,48,1,5,978824351,1,10,1,2,3,0,12


In [206]:
#number of unique zipcode sums .. -- > this is a heuristic approach that may or may not pay off :-) 
len(movies['zipcode_sum'].unique())

27

### *Timestamp*

In [209]:
from datetime import datetime
datetime.fromtimestamp(978824268).strftime("%A, %B %d, %Y %I:%M:%S")

'Saturday, January 06, 2001 11:37:48'

In [210]:
datetime.fromtimestamp(978824268).strftime("%A")

'Saturday'

In [211]:
def weekday(timestamp):
    '''This function returns the weekday from the timestamp'''
    return datetime.fromtimestamp(timestamp).strftime('%A')

In [212]:
#extract the weekday from all the timestamp
movies['weekday'] = movies['Timestamp'].apply(weekday)

In [213]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,zipcode_sum,weekday
0,1,1,5,978824268,1,10,1,2,3,0,12,Saturday
1,48,1,5,978824351,1,10,1,2,3,0,12,Saturday


In [214]:
movies['weekday'].value_counts().sort_values(ascending=False)

Monday       161213
Tuesday      147302
Sunday       133306
Thursday     131551
Wednesday    130305
Friday       119810
Saturday     100402
Name: weekday, dtype: int64

There are maximum reviews on Monday and then followed by Tuesday. These could be the users who might have seen the movies over the weekend. 

In [216]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 923889 entries, 0 to 1000208
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   MovieID      923889 non-null  int64 
 1   UserID       923889 non-null  int64 
 2   Rating       923889 non-null  int64 
 3   Timestamp    923889 non-null  int64 
 4   Age          923889 non-null  int64 
 5   Occupation   923889 non-null  int64 
 6   Count        923889 non-null  int64 
 7   Genre_enc_1  923889 non-null  int64 
 8   Genre_enc_2  923889 non-null  int64 
 9   Gender_enc   923889 non-null  uint8 
 10  zipcode_sum  923889 non-null  int64 
 11  weekday      923889 non-null  object
dtypes: int64(10), object(1), uint8(1)
memory usage: 85.5+ MB


In [217]:
movies['weekday_enc'] = encoder.fit_transform(movies['weekday'])

In [218]:
movies.head(2)

,MovieID,UserID,Rating,Timestamp,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,zipcode_sum,weekday,weekday_enc
0,1,1,5,978824268,1,10,1,2,3,0,12,Saturday,2
1,48,1,5,978824351,1,10,1,2,3,0,12,Saturday,2


In [219]:
movies.drop('weekday',axis=1,inplace=True)

In [220]:
movies.drop('Timestamp',axis=1,inplace=True)

In [221]:
movies.head(3)

,MovieID,UserID,Rating,Age,Occupation,Count,Genre_enc_1,Genre_enc_2,Gender_enc,zipcode_sum,weekday_enc
0,1,1,5,1,10,1,2,3,0,12,2
1,48,1,5,1,10,1,2,3,0,12,2
2,150,1,5,1,10,1,7,7,0,12,3


### *Save the feature engineered dataframe as a separate csv for modeling*

In [222]:
movies.to_csv('movies.csv',index=False)